In [ ]:
# 所有的头

In [2]:
import torch # for save
import os
from van import Fan
from localconfig import config
import datetime
import random
from pprint import pprint
import html
from pyltp import SentenceSplitter
from pyltp import Postagger
from pyltp import Segmentor
from pyltp import NamedEntityRecognizer
import matplotlib.pyplot as plt
from tqdm import tqdm
from IPython.display import Image
from IPython.core.display import HTML 
import pyecharts
from pyecharts import Pie
from pyecharts import Line

%matplotlib inline

plt.rcParams['font.sans-serif']=['FangSong'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号


In [3]:
# 所有的配置

In [4]:
keyword = '贸易战' # 关注的关键词
until_year = 2000 # 最早的年份
until_num = 10000 # 截止的饭否数量
timeout = (60,60) # 超时时间

In [5]:
# 登录
fan = Fan(config['Consumer key'], config['Consumer secret'])
fan.xauth(config['my username'], config['my password'])

{'oauth_token': '1405515-071ad3666b5af9441d5cbb0dc1d3985b',
 'oauth_token_secret': '43cc293f5d127423f960b8636688577b'}

In [6]:
# 本小时内可用API次数
fan.request('GET', 'account/rate_limit_status', {}, timeout=timeout)

{'reset_time': 'Mon Mar 18 08:58:47 +0000 2019',
 'remaining_hits': 150,
 'hourly_limit': 150,
 'reset_time_in_seconds': 1552899527}

In [7]:
# 采集关键词消息
def datetimefromstr(s):
    return datetime.datetime.strptime(s, '%a %b %d %H:%M:%S %z %Y')
def get_status(keyword, until_year, until_num):
    latest = fan.request('GET', 'search/public_timeline', {'q':keyword, 'count':60}, timeout=timeout)
    
    while True:
        year = datetimefromstr(latest[-1]['created_at']).year
        if len(latest) >= until_num:
            break
        if year < until_year:
            break
        print(len(latest), year,'/', until_num, until_year)
        len_ = len(latest)
        latest += fan.request('GET', 'search/public_timeline', { 'q':keyword, 'count':60, 'max_id':latest[-1]['id']}, timeout=timeout)[1:]
        if len(latest) == len_:
            break
    return latest

In [8]:
# 下载并暂存
statuses = get_status(keyword, until_year, until_num)

60 2019 / 10000 2000
119 2019 / 10000 2000
178 2018 / 10000 2000
237 2018 / 10000 2000
296 2018 / 10000 2000
355 2018 / 10000 2000
414 2018 / 10000 2000
473 2018 / 10000 2000
532 2018 / 10000 2000
591 2018 / 10000 2000
650 2018 / 10000 2000
709 2018 / 10000 2000
768 2018 / 10000 2000
827 2018 / 10000 2000
886 2018 / 10000 2000
945 2018 / 10000 2000
1004 2018 / 10000 2000
1063 2018 / 10000 2000
1122 2018 / 10000 2000
1181 2018 / 10000 2000
1240 2018 / 10000 2000
1299 2018 / 10000 2000
1358 2017 / 10000 2000
1417 2012 / 10000 2000
1476 2011 / 10000 2000
1524 2008 / 10000 2000


In [9]:
torch.save(statuses,'tmp_statuses.torch.pkl')

In [10]:
# 恢复
# statuses = torch.load('tmp_statuses.torch.pkl')

In [12]:
print(len(statuses))

1524


In [13]:
# common
def get_top(l, num=5, reverse=False): # 从大到小
    cnt = {}
    for x in l:
        if type(x) == tuple and len(x) == 3:
            x = x[0]
        if x not in cnt:
            cnt[x] = 0
        cnt[x] += 1
    cnt_list = [(k,cnt[k],cnt[k]/len(l)) for k in cnt]
    cnt_list = sorted(cnt_list, key=lambda x:x[1])[::-1]
    if reverse:
        cnt_list = cnt_list[::-1]
    if num == -1:
        return cnt_list
    return cnt_list[:num]
def huabing(y): # x [(label,,0.xxx)]
    attr = [x[0] for x in y]
    v1 = [x[2] for x in y]
    if sum(v1) < 1.0:
        attr.append('其他')
        v1.append(1.0 - sum(v1))
    pie = Pie('')
    pie.add('', attr, v1, is_label_show=True)
    return pie

In [14]:
# 从消息关系本身入手

In [15]:
# 时间段分布

In [16]:
print(datetimefromstr(statuses[-1]['created_at']))
print(datetimefromstr(statuses[0]['created_at']))
begin_year = datetimefromstr(statuses[-1]['created_at']).year
end_year = datetimefromstr(statuses[0]['created_at']).year

2008-04-30 08:59:45+00:00
2019-03-16 01:10:34+00:00


In [17]:
huabing(get_top([datetimefromstr(x['created_at']).year for x in statuses], num = -1))

In [40]:
arr = sorted(get_top([datetimefromstr(x['created_at']).year for x in statuses],num=-1),key = lambda x:x[0])
attr = [str(x[0]) for x in arr ]
v1 = [x[1] for x in arr ]
line = Line(keyword)
line.add('', attr, v1,is_smooth=True,is_label_show=True,mark_line=[ "average"])
# line.add("商家B", attr, v2, is_smooth=True, mark_line=["max", "average"])
line

In [20]:
# 用户信息文件
os.path.exists('./users.torch.pkl')

True

In [21]:
# 预处理用户信息
userid_to_info = {}
if os.path.exists('./users.torch.pkl'):
    userid_to_info = torch.load('users.torch.pkl')
for x in statuses:
    userid_to_info[x['user']['id']] = x['user']
torch.save(userid_to_info, 'users.torch.pkl')
print(len(userid_to_info))

4050


In [22]:
# 发表人数/总消息数量
uni_num = len(set([x['user']['id'] for x in statuses ]))
print(uni_num, len(statuses), uni_num/len(statuses))

669 1524 0.4389763779527559


In [74]:
# 发表top30的人
users_top = get_top([x['user']['id'] for x in statuses], num=30)
names_top = [ (userid_to_info[x[0]]['name'] , x[0], x[1], x[2]) for x in users_top]
pprint(names_top)
huabing(names_top)

[('牛山鎮出品', '牛山镇出品', 49, 0.03215223097112861),
 ('财经频道', 'cashmoney', 34, 0.02230971128608924),
 ('亥乃亚克山冬', 'BillGateFun', 27, 0.017716535433070866),
 ('那.谁', '海边的暴走族', 26, 0.01706036745406824),
 ('抱阳', 'xxlzw', 22, 0.014435695538057743),
 ('MDJsoftware', 'mdjsoftware', 21, 0.013779527559055118),
 ('华尔街日报', '华尔街日报中文网', 19, 0.012467191601049869),
 ('吴下阿阿阿蒙', '~koUOq8qty7I', 18, 0.011811023622047244),
 ('张晓张8586', '~Vr29VIRGRcs', 18, 0.011811023622047244),
 ('安达充的走狗', 'wylwddjz', 17, 0.01115485564304462),
 ('今天你开心吗', 'fyw321', 16, 0.010498687664041995),
 ('时光云海', 'songyuyao', 15, 0.00984251968503937),
 ('饭士', '奋发图强', 14, 0.009186351706036745),
 ('逸博', 'king66lele', 14, 0.009186351706036745),
 ('剑花', 'dezhaos', 14, 0.009186351706036745),
 ('一介大叔', 'ningqun', 13, 0.00853018372703412),
 ('小世界.', 'uramazing', 12, 0.007874015748031496),
 ('Solibot', 'solibot', 11, 0.007217847769028871),
 ('奔驰的回忆', '奔驰的回忆', 11, 0.007217847769028871),
 ('康村的村夫', '~-d7SWGD7osE', 11, 0.007217847769028871),
 ('果然奇异

In [24]:
def show_user_info(x):
    user = userid_to_info[x]
    pprint(user)
    return Image(url = user['profile_image_url_large'])

In [26]:
show_user_info(users_top[0][0])

{'birthday': '0000-10-10',
 'created_at': 'Mon Jul 11 02:28:29 +0000 2011',
 'description': '1 认认真真地活着。\r\n'
                '2 健康工作到六十岁就是人生赢家。\r\n'
                '3 是个比NPC还容易预测的人哪。\r\n'
                '4 你怎么会想起查看我这样无趣的家伙的？',
 'favourites_count': 10,
 'followers_count': 1149,
 'following': False,
 'friends_count': 93,
 'gender': '男',
 'id': '牛山镇出品',
 'location': '海外 美国',
 'name': '牛山鎮出品',
 'notifications': False,
 'photo_count': 895,
 'profile_background_color': '#acdae5',
 'profile_background_image_url': 'http://static.fanfou.com/img/bg/0.png',
 'profile_background_tile': False,
 'profile_image_url': 'http://s3.meituan.net/v1/mss_3d027b52ec5a4d589e68050845611e68/avatar/s0/00/vb/j0.jpg?1489500301',
 'profile_image_url_large': 'http://s3.meituan.net/v1/mss_3d027b52ec5a4d589e68050845611e68/avatar/l0/00/vb/j0.jpg?1489500301',
 'profile_link_color': '#0066cc',
 'profile_sidebar_border_color': '#b2d1a3',
 'profile_sidebar_fill_color': '#e2f2da',
 'profile_text_color': '#222222',
 'protec

In [75]:
# 被引用top20的消息

repost_top = get_top([x['repost_status']['id'] for x in statuses if 'repost_status' in x], num=20)
for i in range(len(repost_top)):
    x = repost_top[i]
    for y in statuses:
        if 'repost_status' in y and y['repost_status']['id'] == x[0]:
            repost_top[i] = ('id: '+y['repost_status']['user']['id'] + ' name: '+y['repost_status']['user']['name']  +' '+ y['repost_status']['text'],x[0], x[1],x[2])
            break
            
pprint(repost_top)

[('id: 海边的暴走族 name: 那.谁 未来的贸易战，贵国被打成日本也不是不可能，自求多福吧。',
  'B0MzT8DTMEw',
  14,
  0.03218390804597701),
 ('id: sexyfish name: sexyfish '
  '最近发现了一个节奏：贸易战--&gt;国库告急--&gt;催缴税款--&gt;舆论围剿（各个灰色行业）---&gt;税务出马--&gt;巨额罚款',
  'XrHuLxwkGKU',
  14,
  0.03218390804597701),
 ('id: xxlzw name: 抱阳 '
  '如果不是因为一个叫特朗普的疯子发起的贸易战，我们大概率会陶醉在过去40年的“经济奇迹”与“大国崛起”的耀眼光环里，感觉良好，乃至一醉不起',
  'MavNsbPgbFY',
  11,
  0.02528735632183908),
 ('id: uramazing name: 小世界. '
  '下周贸易战2000亿出结果，如果美国对我们进行贸易战的话，在中国，就是这个经济发展的结束了。因为我们长期的发展，基本上都是开放人力，借助外力来发展的。如果美国跟我们打贸易战，增长型的经济不会出现了，那么只会出现所谓存量的经济的争夺战。存量经济的争夺本质上都是财富转移而已。',
  '06vPmkJNdNc',
  11,
  0.02528735632183908),
 ('id: wangchu name: Tennichy 中国已经在仍未结束的贸易战中赢了一百多次了。',
  'CAWAVl666pM',
  9,
  0.020689655172413793),
 ('id: 牛山镇出品 name: 牛山鎮出品 '
  '【转】2018年4月，湖南大学研究生辩论赛决赛，正方观点为“掀起中美贸易战是因为美国膨胀了”，反方观点是“掀起中美贸易战是因为美国心虚了”。--也不应该忽视其他可能啊，譬如“两者皆是”:-)',
  'DMERRcLwpSQ',
  8,
  0.01839080459770115),
 ('id: gravity0 name: gravity0 搜索出口退税（export tax '
  'refund），出来结果几乎都是关于天朝，貌似美帝没有相应政策。理论上如果天朝把增收的进口税全

In [76]:
# 引用top20的人
repost_user_id_top = get_top([x['repost_status']['user']['id'] for x in statuses if 'repost_status' in x], num=20)
repost_name_top =  [ (userid_to_info[x[0]]['name'] , x[0], x[1], x[2]) for x in repost_user_id_top]
pprint(repost_name_top)

[('牛山鎮出品', '牛山镇出品', 56, 0.12873563218390804),
 ('sexyfish', 'sexyfish', 21, 0.04827586206896552),
 ('抱阳', 'xxlzw', 18, 0.041379310344827586),
 ('那.谁', '海边的暴走族', 17, 0.03908045977011494),
 ('小世界.', 'uramazing', 16, 0.0367816091954023),
 ('剑花', 'dezhaos', 14, 0.03218390804597701),
 ('康村的村夫', '~-d7SWGD7osE', 13, 0.029885057471264367),
 ('Tennichy', 'wangchu', 12, 0.027586206896551724),
 ('亥乃亚克山冬', 'BillGateFun', 12, 0.027586206896551724),
 ('gravity0', 'gravity0', 9, 0.020689655172413793),
 ('王兴', 'wangxing', 9, 0.020689655172413793),
 ('饭而不否', 'dyxin', 9, 0.020689655172413793),
 ('安达充的走狗', 'wylwddjz', 8, 0.01839080459770115),
 ('慧淫禅师', '~9EKpNhzk6yw', 7, 0.016091954022988506),
 ('二他姐姐', 'znz28', 7, 0.016091954022988506),
 ('饭士', '奋发图强', 6, 0.013793103448275862),
 ('瑞秋Rachel', 'sugarball86', 6, 0.013793103448275862),
 ('一介大叔', 'ningqun', 6, 0.013793103448275862),
 ('素合子', 'suheti', 5, 0.011494252873563218),
 ('猫爹', 'imAdam', 5, 0.011494252873563218)]


In [43]:
# top20 gender
gender_top = get_top([x['user']['gender'] for x in statuses ], num=20)
pprint(gender_top)
huabing(gender_top)

[('男', 843, 0.5531496062992126),
 ('', 473, 0.31036745406824146),
 ('女', 208, 0.13648293963254593)]


In [44]:
# top10 location
location_top10 = get_top
location_top10 = get_top([x['location'].split()[0] for x in statuses if len(x['location'].split())>=1], num=10)
pprint(location_top10)
huabing(location_top10)

[('北京', 173, 0.17545638945233266),
 ('广东', 130, 0.13184584178498987),
 ('上海', 108, 0.10953346855983773),
 ('美国', 60, 0.060851926977687626),
 ('山东', 50, 0.05070993914807302),
 ('江苏', 40, 0.04056795131845842),
 ('四川', 35, 0.035496957403651115),
 ('浙江', 34, 0.034482758620689655),
 ('河南', 32, 0.032454361054766734),
 ('重庆', 25, 0.02535496957403651)]


In [45]:
def remove_atag(s):
    if '<a' not in s:
        return s
    start = s.find('>')
    end = s.find('</a>',start)
    return s[start+1:end]

In [46]:
# top20 source
source_top = get_top([remove_atag(x['source']) for x in statuses ], num=20)
pprint(source_top)
huabing(source_top)

[('网页', 478, 0.3136482939632546),
 ('御飯 iOS', 229, 0.15026246719160105),
 ('猫饭', 134, 0.08792650918635171),
 ('蘑菇饭', 108, 0.07086614173228346),
 ('Android客户端', 69, 0.045275590551181105),
 ('尚饭', 66, 0.04330708661417323),
 ('手机上网', 58, 0.03805774278215223),
 ('有饭', 50, 0.03280839895013123),
 ('API', 41, 0.02690288713910761),
 ('饭否财经频道', 34, 0.02230971128608924),
 ('炒饭', 32, 0.02099737532808399),
 ('饭团fantuan', 26, 0.01706036745406824),
 ('享拍微博通', 25, 0.016404199475065617),
 ('小饭', 22, 0.014435695538057743),
 ('饭否分享', 20, 0.013123359580052493),
 ('泡饭', 19, 0.012467191601049869),
 ('FFReeder', 17, 0.01115485564304462),
 ('Twidere', 14, 0.009186351706036745),
 ('Mr.Greeting', 11, 0.007217847769028871),
 ('FaWave', 8, 0.005249343832020997)]


In [47]:
# nlp part

In [48]:
def myfilter(s):
    s = s.replace('<strong>','')
    s = s.replace('</strong>','')
    while '转@' in s:
        pos = s.find('转@')
        end = len(s)
        for endsym in [' ', ']', '：',':','@']:
            end2 = s.find(endsym,pos+2)
            if end2 == -1:
                end2 = len(s)
            end = min(end,end2)
        if end-pos > 15:
            print('alert [转@] too long',s)
            s = s[:pos] + s[pos+2:]
        else:
            s = s[:pos] + s[end:]
    while '@' in s:
        pos = s.find('@')
        end = len(s)
        for endsym in [' ', ']', '：',':','@']:
            end2 = s.find(endsym,pos+1)
            if end2 == -1:
                end2 = len(s)
            end = min(end,end2)
        if end-pos > 15:
            print('alert [@] too long',s)
            s = s[:pos] + s[pos+1:]
        else:
            s = s[:pos] + s[end:]
    if '@' in s:
        print('alert @ in ', s)
    return s

In [49]:
sentences = []
for x in tqdm(statuses):
    s = html.unescape(x['text']) 
    s = myfilter(s)
    if keyword not in s:
        continue
    if '</' in s:
        print(s)
    ss = SentenceSplitter.split(s)
    for t in ss:
        if len(t) > 4:
            sentences.append(t)
    

100%|██████████| 1524/1524 [00:00<00:00, 9702.32it/s]

alert [@] too long @鋆言天下
犹他州将实行全国最严格的酒驾法规，凡血液酒精含量达.05%就属酒驾。这个虽然不是坏事但其实犹他州药驾的人还不少，经常看到大白天有驾车人蛇形开车的。/川普推特表示将支付第二笔款补贴给受中美贸易战影响的农民。

借着中国的外债，跟中国打着贸易战，再补贴受对华贸易战争伤害的
alert [@] too long @旅欧村民
我越来越相信川普的认知是，中国经济越坏对美国来说越好。另外，他对于自己没有过或者不会有投资利益的地方不care，但对他家有过或者有利益的地方绝对不会打击。俄国、沙特就是例子。如果川家在中国有投资的话，他很可能不会发起贸易战。 ​​​​
alert [@] too long @小世界.跟一个国家一样，不是贸易战，我们会发生改变吗？人总是遇到外力，才会发生转变。没有外力的时候，自己怎么做都觉得是对的。只有外力的强加，才会有重新调整迭代的机会。有时候需要示弱，有时候需要示强。人一生总要经历，七灾八劫，这也是成长的阶梯，并且在这个磨砺的过程中，让自己调配到


In [50]:
print(len(sentences))

3345


In [51]:
# 对sentences去重。。 拒绝转发的影响 （存疑？）
# 不去重

In [52]:
# sentences = list(set(sentences))

In [53]:
print(len(list(set(sentences))))

2580


In [54]:
# torch.save(['尼玛', '卧槽', '傻逼', '对了', '有木有', '他妈'],'blacklist.torch.pkl' )
def add_to_blacklist(w):
    global blacklist
    blacklist = torch.load('blacklist.torch.pkl')
    blacklist.append(w)
    torch.save(blacklist,'blacklist.torch.pkl')

In [55]:
# 无意义
blacklist = torch.load('blacklist.torch.pkl')

In [56]:
words_all = []

segmentor = Segmentor()
postagger = Postagger()
recognizer = NamedEntityRecognizer()
postagger.load('./ltp_data_v3.4.0/pos.model')
segmentor.load('./ltp_data_v3.4.0/cws.model')
recognizer.load('./ltp_data_v3.4.0/ner.model')

#todo 情感分析

for s in tqdm(sentences):
    words = segmentor.segment(s)
    postags = postagger.postag(words) #词性标注
    netags = recognizer.recognize(words, postags) # 命名实体识别
    for i in range(len(words)):
        if words[i] in blacklist:
            continue
        words_all.append((words[i],postags[i],netags[i]))

segmentor.release()
postagger.release()
recognizer.release()

100%|██████████| 3345/3345 [00:05<00:00, 642.05it/s]


In [57]:
print(len(words_all))

66265


In [58]:
get_top([x[0] for x in words_all if len(x[0])>1 and x[1]!='wp'], num = 25)

[('贸易战', 1469, 0.046896948027071894),
 ('中国', 760, 0.02426254629038437),
 ('美国', 692, 0.022091686885455243),
 ('贸易', 386, 0.012322819563274167),
 ('经济', 258, 0.00823649597752522),
 ('我们', 244, 0.00778955433533393),
 ('因为', 163, 0.005203677691227174),
 ('可能', 162, 0.005171753288213511),
 ('一个', 159, 0.005075980079172519),
 ('如果', 122, 0.003894777167666965),
 ('没有', 118, 0.00376707955561231),
 ('影响', 114, 0.0036393819435576553),
 ('这个', 104, 0.003320137913421019),
 ('现在', 102, 0.003256289107393692),
 ('川普', 101, 0.0032243647043800282),
 ('美元', 97, 0.0030966670923253735),
 ('国家', 94, 0.0030008938832843824),
 ('什么', 91, 0.0029051206742433918),
 ('特朗普', 89, 0.0028412718682160644),
 ('还是', 88, 0.0028093474652024007),
 ('出口', 88, 0.0028093474652024007),
 ('关税', 84, 0.002681649853147746),
 ('新闻', 84, 0.002681649853147746),
 ('欧盟', 83, 0.0026497254501340827),
 ('日本', 82, 0.002617801047120419)]

In [59]:
posdict = {}
nedict = {}
for x in tqdm(words_all):
    if x[1] not in posdict:
        posdict[x[1]] = []
    if x[2] not in nedict:
        nedict[x[2]] = []
    posdict[x[1]].append(x)
    nedict[x[2]].append(x)

100%|██████████| 66265/66265 [00:00<00:00, 868437.34it/s]


In [60]:
# 词性
print(posdict.keys())


dict_keys(['nt', 'v', 'u', 'q', 'n', 'wp', 'ni', 'd', 'nl', 'i', 'r', 'o', 'ns', 'j', 'm', 'a', 'nz', 'p', 'nh', 'nd', 'ws', 'c', 'b', 'e', 'z', 'k', 'h'])


In [61]:
# ['i'] 成语
pprint(get_top(posdict['i'], num=25))

[('没什么', 11, 0.026378896882494004),
 ('一醉不起', 10, 0.023980815347721823),
 ('亲朋好友', 9, 0.02158273381294964),
 ('一触即发', 8, 0.019184652278177457),
 ('说不定', 8, 0.019184652278177457),
 ('经济危机', 8, 0.019184652278177457),
 ('好不了', 7, 0.016786570743405275),
 ('两败俱伤', 6, 0.014388489208633094),
 ('他妈的', 6, 0.014388489208633094),
 ('惨绿', 5, 0.011990407673860911),
 ('急刹车', 5, 0.011990407673860911),
 ('风平浪静', 5, 0.011990407673860911),
 ('一厢情愿', 5, 0.011990407673860911),
 ('悬崖勒马', 4, 0.009592326139088728),
 ('对外开放', 4, 0.009592326139088728),
 ('有意思', 4, 0.009592326139088728),
 ('一泻千里', 4, 0.009592326139088728),
 ('摩拳擦掌', 4, 0.009592326139088728),
 ('一败涂地', 4, 0.009592326139088728),
 ('就地取材', 4, 0.009592326139088728),
 ('不可避免', 4, 0.009592326139088728),
 ('畅通无阻', 3, 0.007194244604316547),
 ('空穴来风', 3, 0.007194244604316547),
 ('无心插柳', 3, 0.007194244604316547),
 ('闭关锁国', 3, 0.007194244604316547)]


In [62]:

# ['j'] 缩略词
pprint(get_top(posdict['j'], num=25))


[('美', 528, 0.3550773369199731),
 ('中', 450, 0.3026227303295225),
 ('欧盟', 83, 0.05581708137188971),
 ('欧', 61, 0.0410221923335575),
 ('华', 26, 0.017484868863483525),
 ('川普', 21, 0.014122394082044385),
 ('房地产', 19, 0.012777404169468728),
 ('央行', 19, 0.012777404169468728),
 ('俄', 15, 0.010087424344317418),
 ('韩', 9, 0.00605245460659045),
 ('日', 9, 0.00605245460659045),
 ('微博', 8, 0.005379959650302623),
 ('中共', 8, 0.005379959650302623),
 ('发改委', 8, 0.005379959650302623),
 ('经贸', 8, 0.005379959650302623),
 ('央视', 7, 0.004707464694014795),
 ('华大', 6, 0.004034969737726967),
 ('官媒', 6, 0.004034969737726967),
 ('贸易战', 6, 0.004034969737726967),
 ('政协', 6, 0.004034969737726967),
 ('世贸', 5, 0.0033624747814391394),
 ('三反五反', 5, 0.0033624747814391394),
 ('川', 5, 0.0033624747814391394),
 ('人大', 5, 0.0033624747814391394),
 ('汽协', 4, 0.0026899798251513113)]


In [63]:

# ['n'] 名词
pprint(get_top(posdict['n'], num=25))


[('贸易战', 1111, 0.09641586392432526),
 ('人', 268, 0.023257832161763428),
 ('经济', 258, 0.022390002603488675),
 ('国家', 94, 0.008157597847782696),
 ('关税', 84, 0.0072897682895079405),
 ('新闻', 84, 0.0072897682895079405),
 ('问题', 76, 0.006595504642888137),
 ('时候', 75, 0.006508721687060661),
 ('战', 73, 0.00633515577540571),
 ('国', 71, 0.00616158986375076),
 ('公司', 68, 0.005901240996268333),
 ('政府', 66, 0.005727675084613382),
 ('美帝', 62, 0.00538054326130348),
 ('国际', 60, 0.005206977349648529),
 ('大豆', 60, 0.005206977349648529),
 ('结果', 57, 0.004946628482166103),
 ('市场', 55, 0.004773062570511152),
 ('股', 52, 0.004512713703028725),
 ('人民币', 51, 0.0044259307472012495),
 ('股市', 50, 0.0043391477913737745),
 ('企业', 49, 0.004252364835546299),
 ('世界', 47, 0.004078798923891348),
 ('产品', 45, 0.003905233012236397),
 ('战争', 44, 0.003818450056408921),
 ('全球', 44, 0.003818450056408921)]


In [64]:

# ['nh'] 人名词
pprint(get_top(posdict['nh'], num=25))


[('特朗普', 27, 0.06192660550458716),
 ('奥巴马', 26, 0.05963302752293578),
 ('川普', 24, 0.05504587155963303),
 ('孟晚舟', 16, 0.03669724770642202),
 ('蛤蛤', 15, 0.034403669724770644),
 ('罗姆尼', 8, 0.01834862385321101),
 ('崔永元', 8, 0.01834862385321101),
 ('陈德铭', 7, 0.016055045871559634),
 ('崔天凯', 7, 0.016055045871559634),
 ('张首晟', 6, 0.013761467889908258),
 ('胡锡进', 5, 0.011467889908256881),
 ('托腮', 4, 0.009174311926605505),
 ('孙立平', 4, 0.009174311926605505),
 ('董扬', 4, 0.009174311926605505),
 ('冯小刚', 4, 0.009174311926605505),
 ('许元', 4, 0.009174311926605505),
 ('老美', 4, 0.009174311926605505),
 ('基辛格', 4, 0.009174311926605505),
 ('里根', 4, 0.009174311926605505),
 ('刘鹤', 4, 0.009174311926605505),
 ('洪博培', 3, 0.006880733944954129),
 ('•罗奇', 3, 0.006880733944954129),
 ('斯蒂芬', 3, 0.006880733944954129),
 ('李克强', 3, 0.006880733944954129),
 ('王', 3, 0.006880733944954129)]


In [65]:

# ['ni'] 组织名
pprint(get_top(posdict['ni'], num=25))


[('路透社', 5, 0.22727272727272727),
 ('外交部', 4, 0.18181818181818182),
 ('国务院', 3, 0.13636363636363635),
 ('北京大学', 2, 0.09090909090909091),
 ('WTO', 2, 0.09090909090909091),
 ('财政部', 2, 0.09090909090909091),
 ('联合国', 1, 0.045454545454545456),
 ('中国银行', 1, 0.045454545454545456),
 ('透社', 1, 0.045454545454545456),
 ('中共中央', 1, 0.045454545454545456)]


In [66]:

# ['nz'] other proper name
pprint(get_top(posdict['nz'], num=25))


[('华为', 48, 0.1509433962264151),
 ('中兴', 45, 0.14150943396226415),
 ('川普', 45, 0.14150943396226415),
 ('思科', 15, 0.04716981132075472),
 ('人民日报', 13, 0.040880503144654086),
 ('腾讯', 11, 0.03459119496855346),
 ('英语', 11, 0.03459119496855346),
 ('WTO', 10, 0.031446540880503145),
 ('中华', 10, 0.031446540880503145),
 ('腾讯网', 8, 0.025157232704402517),
 ('英文', 7, 0.0220125786163522),
 ('搜狐', 6, 0.018867924528301886),
 ('鸦片战争', 5, 0.015723270440251572),
 ('联想', 5, 0.015723270440251572),
 ('网易', 4, 0.012578616352201259),
 ('卡内基', 3, 0.009433962264150943),
 ('康师傅', 3, 0.009433962264150943),
 ('高通', 3, 0.009433962264150943),
 ('星巴克', 3, 0.009433962264150943),
 ('优衣库', 3, 0.009433962264150943),
 ('富力', 3, 0.009433962264150943),
 ('美团', 2, 0.006289308176100629),
 ('麦当劳', 2, 0.006289308176100629),
 ('安卓', 2, 0.006289308176100629),
 ('阿里巴巴', 2, 0.006289308176100629)]


In [67]:

# ['v'] verb len >= 2
pprint(get_top([x for x in posdict['v'] if len(x[0])>=2], num=25))


[('贸易', 386, 0.04411428571428572),
 ('贸易战', 352, 0.040228571428571426),
 ('可能', 145, 0.01657142857142857),
 ('影响', 114, 0.01302857142857143),
 ('出口', 88, 0.010057142857142857),
 ('可以', 79, 0.00902857142857143),
 ('觉得', 76, 0.008685714285714286),
 ('发展', 72, 0.008228571428571429),
 ('开始', 71, 0.008114285714285715),
 ('出现', 66, 0.007542857142857143),
 ('没有', 65, 0.0074285714285714285),
 ('进口', 63, 0.0072),
 ('表示', 53, 0.006057142857142857),
 ('还有', 52, 0.005942857142857143),
 ('导致', 52, 0.005942857142857143),
 ('应该', 50, 0.005714285714285714),
 ('结束', 50, 0.005714285714285714),
 ('知道', 50, 0.005714285714285714),
 ('看到', 46, 0.005257142857142857),
 ('引发', 44, 0.005028571428571428),
 ('认为', 43, 0.004914285714285714),
 ('感觉', 38, 0.004342857142857143),
 ('开放', 38, 0.004342857142857143),
 ('爆发', 36, 0.004114285714285714),
 ('需要', 36, 0.004114285714285714)]


In [69]:

# foreign words	
pprint(get_top([x for x in posdict['ws'] if len(x[0])>=4 and not x[0].startswith('http://')], num=25))


[('refund', 8, 0.0427807486631016),
 ('export', 8, 0.0427807486631016),
 ('Twitter', 7, 0.0374331550802139),
 ('wellness', 7, 0.0374331550802139),
 ('Ecup', 6, 0.03208556149732621),
 ('caojitw', 5, 0.026737967914438502),
 ('williamlong', 5, 0.026737967914438502),
 ('list', 4, 0.0213903743315508),
 ('iPhone', 4, 0.0213903743315508),
 ('ICAO', 3, 0.016042780748663103),
 ('license', 3, 0.016042780748663103),
 ('news', 3, 0.016042780748663103),
 ('fake', 3, 0.016042780748663103),
 ('Tabgach', 3, 0.016042780748663103),
 ('lose', 2, 0.0106951871657754),
 ('someone', 2, 0.0106951871657754),
 ('know', 2, 0.0106951871657754),
 ('Boss', 2, 0.0106951871657754),
 ('even', 2, 0.0106951871657754),
 ("can't", 2, 0.0106951871657754),
 ('wrong', 2, 0.0106951871657754),
 ('freedom', 2, 0.0106951871657754),
 ('philo', 2, 0.0106951871657754),
 ('apple', 2, 0.0106951871657754),
 ('Metoo', 2, 0.0106951871657754)]


In [70]:
# 命名实体
print(nedict.keys())


dict_keys(['O', 'S-Ni', 'S-Ns', 'S-Nh', 'B-Ni', 'E-Ni', 'I-Ni', 'B-Ns', 'I-Ns', 'E-Ns', 'B-Nh', 'E-Nh'])


In [71]:


# 人名
pprint(get_top(nedict['S-Nh'], num=25))


[('特朗普', 27, 0.07068062827225131),
 ('奥巴马', 25, 0.06544502617801047),
 ('川普', 24, 0.06282722513089005),
 ('孟晚舟', 16, 0.041884816753926704),
 ('罗姆尼', 8, 0.020942408376963352),
 ('崔永元', 8, 0.020942408376963352),
 ('陈德铭', 7, 0.01832460732984293),
 ('崔天凯', 7, 0.01832460732984293),
 ('张首晟', 6, 0.015706806282722512),
 ('胡锡进', 5, 0.013089005235602094),
 ('托腮', 4, 0.010471204188481676),
 ('孙立平', 4, 0.010471204188481676),
 ('董扬', 4, 0.010471204188481676),
 ('冯小刚', 4, 0.010471204188481676),
 ('许元', 4, 0.010471204188481676),
 ('老美', 4, 0.010471204188481676),
 ('基辛格', 4, 0.010471204188481676),
 ('里根', 4, 0.010471204188481676),
 ('刘鹤', 4, 0.010471204188481676),
 ('洪博培', 3, 0.007853403141361256),
 ('李克强', 3, 0.007853403141361256),
 ('王', 3, 0.007853403141361256),
 ('蔡英文', 3, 0.007853403141361256),
 ('老胡', 3, 0.007853403141361256),
 ('希特勒', 3, 0.007853403141361256)]


In [72]:

# 机构名
pprint(get_top(nedict['S-Ni'], num=25))


[('路透社', 4, 0.25),
 ('国务院', 2, 0.125),
 ('外交部', 2, 0.125),
 ('WTO', 2, 0.125),
 ('财政部', 2, 0.125),
 ('联合国', 1, 0.0625),
 ('中国银行', 1, 0.0625),
 ('透社', 1, 0.0625),
 ('中共中央', 1, 0.0625)]


In [73]:

# 地名
pprint(get_top(nedict['S-Ns'], num=25))

[('中国', 712, 0.24176570458404076),
 ('美国', 655, 0.22241086587436332),
 ('美', 527, 0.17894736842105263),
 ('中', 450, 0.15280135823429541),
 ('日本', 77, 0.026146010186757215),
 ('欧', 61, 0.02071307300509338),
 ('俄罗斯', 27, 0.009168081494057725),
 ('欧洲', 24, 0.008149405772495755),
 ('加拿大', 19, 0.0064516129032258064),
 ('俄', 15, 0.0050933786078098476),
 ('非洲', 14, 0.004753820033955857),
 ('山东', 14, 0.004753820033955857),
 ('伊朗', 14, 0.004753820033955857),
 ('韩国', 13, 0.004414261460101867),
 ('朝鲜', 12, 0.0040747028862478775),
 ('英国', 12, 0.0040747028862478775),
 ('香港', 12, 0.0040747028862478775),
 ('北京', 10, 0.003395585738539898),
 ('南海', 9, 0.0030560271646859084),
 ('土耳其', 9, 0.0030560271646859084),
 ('日', 9, 0.0030560271646859084),
 ('韩', 8, 0.0027164685908319186),
 ('台湾', 8, 0.0027164685908319186),
 ('黄海', 8, 0.0027164685908319186),
 ('古巴', 8, 0.0027164685908319186)]


In [ ]:
# 以上内容均为自动生成...